In [ ]:
import unittest
from unittest.mock import Mock, patch

class TestVespaManager(unittest.TestCase):
    def setUp(self):
        """Setup basic test fixtures"""
        self.schema_id = "test_schema"
        self.env = "test_env"
        self.s3_bucket_id = "test_bucket"
        self.creds = {"key": "value"}
        self.logging_dir = "test_dir"
        
        # Simple mock for S3
        self.mock_s3 = Mock()
        
        # Create VespaManager instance
        with patch('vespa_manager.S3', return_value=self.mock_s3):
            self.manager = VespaManager(
                self.schema_id,
                self.env,
                self.s3_bucket_id,
                self.creds,
                self.logging_dir
            )

    def test_move_to_permanent_failures(self):
        """Test moving a failed payload to permanent failures"""
        # Test data
        native_id = "test123"
        payload = {"data": "test_data"}
        
        # Run the function
        self.manager.move_to_permanent_failures(native_id, payload)
        
        # Check if S3 write_file was called correctly
        expected_path = f"{self.schema_id}/{self.logging_dir}/permanent_failures/{native_id}.json"
        self.mock_s3.write_file.assert_called_once()
        call_args = self.mock_s3.write_file.call_args[0]
        self.assertEqual(call_args[0], expected_path)

    def test_get_failed_payloads_empty(self):
        """Test getting failed payloads when none exist"""
        # Setup mock
        self.mock_s3.file_exists.return_value = False
        
        # Run the function
        result = self.manager.get_failed_payloads()
        
        # Verify result
        self.assertEqual(result, {})

    def test_get_failed_payloads_with_data(self):
        """Test getting failed payloads when they exist"""
        # Setup mocks
        self.mock_s3.file_exists.return_value = True
        self.mock_s3.read_file.return_value = '{"id1": {"status": "failed"}}'
        
        # Run the function
        result = self.manager.get_failed_payloads()
        
        # Verify result
        self.assertEqual(result, {"id1": {"status": "failed"}})

if __name__ == '__main__':
    unittest.main()